# Distance ratio, look-up table, and application

In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%cd D:\FlowsGeneration

D:\FlowsGeneration


In [3]:
import pandas as pd
import scipy.interpolate

## 1. Load distance ratio
Distance ratio (ratio) = Travel distance or network distance / straight line distance or Euclidean distance

### 1.1 Travel survey based distance ratio
Travel surveys of Sweden and the Netherlands contain participant-reported travel distance and the trip distance is calculated as the straight-line distance (Euclidean distance) between centroids of statistics zones, e.g., DeSO zones for Sweden.

In [4]:
df_data = pd.read_csv('results/ratio_data.csv')
df_data.head()

,distance,ratio,lower_ratio,upper_ratio,country
0,0.121503,2.473143,2.372895,2.573390,sweden
1,0.187461,3.550169,1.840718,9.700040,sweden
2,0.287628,2.805004,1.709243,4.621175,sweden
3,0.414068,2.088832,1.285247,3.540168,sweden
4,0.590090,2.016771,1.548158,3.121659,sweden


### 1.2 Simulation-based distance ratio
A simulation for ten selected urban regions whose area varies between 272.3 -- 4878.7 in km$^2$ is conducted to see how much we underestimate the travel distance in urban areas. We use the model-synthesised trips within these urban areas and calculate the shortest network distance assuming driving plus 400 m walking (5 min) using the drive road networks downloaded from OpenStreetMap. The distance ratio decreases along with the trip distance and such a pattern is similar across all the ten urban areas. This opens up the possibility to create a multiplier function of trip distance that compensates for the underestimated travel distance.

More details is [here](https://research.chalmers.se/en/publication/523815).

In [5]:
df_simulation = pd.read_csv('results/ratio_simulation.csv')
df_simulation.head()

,distance,ratio,lower_ratio,upper_ratio
0,0.120453,4.796603,4.085980,6.072655
1,0.162514,3.966736,3.434608,4.852263
2,0.218579,3.293649,2.887725,4.108657
3,0.295120,2.809982,2.481010,3.475213
4,0.401855,2.421351,2.161574,2.949358


## 2. Create lookup tables by interpolation

In [10]:
sp_data_sweden = scipy.interpolate.interp1d(df_data.loc[df_data.country == 'sweden', ['distance']].values.reshape(-1),
                                            df_data.loc[df_data.country == 'sweden', ['ratio']].values.reshape(-1))
sp_data_netherlands = scipy.interpolate.interp1d(df_data.loc[df_data.country == 'netherlands', ['distance']].values.reshape(-1),
                                            df_data.loc[df_data.country == 'netherlands', ['ratio']].values.reshape(-1))
sp_data_simulation = scipy.interpolate.interp1d(df_simulation.loc[:, ['distance']].values.reshape(-1),
                                                df_simulation.loc[:, ['ratio']].values.reshape(-1))

## 3. Use the lookup tables to get a distance ratio given its trip distance

In [12]:
df_test = pd.DataFrame()
test_distances = [1, 2, 3, 4, 5, 10, 15, 78, 200] # km
df_test.loc[:, 'trip distance'] = test_distances
df_test.loc[:, 'ratio (sweden)'] = sp_data_sweden(test_distances)
df_test.loc[:, 'ratio (netherlands)'] = sp_data_netherlands(test_distances)
df_test.loc[:, 'ratio (simulation)'] = sp_data_simulation(test_distances)
df_test

,trip distance,ratio (sweden),ratio (netherlands),ratio (simulation)
0,1,1.960368,2.000592,1.740077
1,2,1.648406,1.653043,1.484936
2,3,1.599986,1.552652,1.396144
3,4,1.539345,1.457330,1.353066
4,5,1.528590,1.453194,1.327054
5,10,1.434981,1.424680,1.263294
6,15,1.397148,1.408961,1.228804
7,78,1.274283,1.293951,1.201010
8,200,1.250330,1.235738,1.174810
